In [1042]:
import Pkg
Pkg.activate(".")

In [1043]:
using Plots

# Optimization & Root finding

1. Root finding: find $x$ such that $f(x)=0$
    - Newton-Raphson (see below)
    - more generally: gradient descend $\subset$ linesearch 
    - Bisection: gradient free
2. Optimization: find $\argmin_x f(x)$
    - Constraint or unconstraint
    - Local or global
        - local is "easy", follow the gradient
        - global is typically hard. Function may have _many_ local minima. Finding the global one may require searching parameter space exhaustively.
    - with or with Jacobian
3. If $f$ is differentiable, then local optimization is equivalent to finding roots of $f'$.
    - gradient methods require _Hessian_ (or some approximation) of $f$.
        - Famous example: _Newton-Raphson_: $x_{k+1} = x_k - f'(x_k)/f''(x_k) $
        - Approximate Hessian: Quasi-Newton, i.e. BFGS
4. Vice-versa: Read the root finding problem as $\argmin_x \Vert f(x) \Vert^2 $

# Root finding

`Roots.jl`: derivative (free) and bisection methods for scalar, univariate functions

In [ ]:
import Roots

In [ ]:
f(x) = exp(-x^2) - x

In [ ]:
Roots.find_zero(f, 5.0)

In [ ]:
Roots.find_zeros(sin, -4π, 4π) ./ π

`NLsolve`: (Quasi-)Newton, Trust-Region, Linesearch for multivariate function

In [ ]:
import NLsolve

In [ ]:
μ = randn(10);
F(x) = 1 - exp(-sum(abs2, (x.-μ)) )

In [ ]:
Fopt = NLsolve.nlsolve(F, zeros(10), iterations=10000, autoscale=false, show_trace=false, extended_trace=false)

In [ ]:
[Fopt.zero μ]

# Optimization

## Least square methods

Common task: Given data $(x,y)$, find parameters such that $\Vert f(x;p) - y \Vert^2_2$ is minimal.

Often $f(x;p) = Xp$ is a linear problem. If noise is Gaussian, then least-squares is max-likelihood.

### `GLM` - (Generalized) Linear Models

Linear, logistic, probit, ... regression

In [ ]:
using DataFrames, GLM

In [ ]:
n = 20

In [ ]:
data = DataFrame(x1=range(0,2π,length=n), x2=range(-π,π,length=n))

data.y = data.x1*1 + data.x2*2 + 0.2*randn(length(data.x1));

In [ ]:
x = Array(data[:,1:2]);

In [ ]:
first(data, 3)

In [ ]:
lm(@formula(y ~ x1+x2+1), data)

### `LsqFit` - Levenberg-Marquardt

Variant of Gauss-Newton with regularisation. Requires gradient.

In [ ]:
using LsqFit

In [ ]:
data.z = sin.(data.x1*1 + data.x2*2) .+ 0.1*randn(length(data.x1));

Minimize $ \sum_i (f_i(x;p)i - z_i)^2 $

Trial functions $f(x;p) \sim \sin(x\cdot p_1) + \cos(x\cdot p_2) + 1$

In [ ]:
x

In [ ]:
f(x, p) = p[1].*sin.(p[2].*x[:,1] + p[3]*x[:,2]) .+  p[4].*cos.(p[5].*x[:,1] + p[6]*x[:,2]) .+ p[7]

In [ ]:
f(x, ones(7))

In [ ]:
ffit = LsqFit.curve_fit(f, x, data.z, ones(7))

In [ ]:
println("L2 residue: ",sum(abs2, ffit.resid))

In [ ]:
ffit.param

### Optim

In [ ]:
using Optim

In [ ]:
optim_res = Optim.optimize(p->sum(abs2, f(x, p).-data.z), ones(7), NewtonTrustRegion())

In [ ]:
[optim_res.minimizer ffit.param]

In [ ]:
optim_res_sa = Optim.optimize(p->sum(abs2, f(x, p).-data.z), ones(7), NelderMead(intial_simplex=Optim.AffineSimplexer(1.0,7.0)), Optim.Options(iterations=10000))

In [ ]:
optim_res_sa.minimizer

### BlackBoxOptim

In [ ]:
import BlackBoxOptim

In [ ]:
BlackBoxOptim.bboptimize(p->sum(abs2, f(x, p).-data.z), SearchRange=fill((-0.1,5.0), 7), MaxSteps=1000_000, PopulationSize=200)

### JuMP

Modeling language for optimization problems.
Supports many different solvers with a unifying syntax.

In [ ]:
using JuMP, Ipopt

In [ ]:
jump_model = Model(Ipopt.Optimizer)

In [ ]:
@variable(jump_model, p[1:7], start=1.0)

In [ ]:
delta = @NLexpression(jump_model, delta[i=1:n], (p[1]*sin(x[i,1]*p[2]+x[i,2]*p[3]) + p[4]*cos(x[i,1]*p[5]+x[i,2]*p[6]) + p[7]) - data.z[i]);

In [ ]:
@NLobjective(jump_model, Min, sum(delta[i]^2 for i=1:n))

In [ ]:
@constraint(jump_model, p[4]==0)

In [ ]:
print(jump_model)

In [ ]:
JuMP.optimize!(jump_model)

In [ ]:
[value.(p) ffit.param]

#### Ising

Find groundstate of random coupling Ising model using M(ixed)I(nteger)N(on)L(inear)P(rogramming) (_MINLP_)

In [ ]:
using Juniper

In [ ]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)

ising = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

In [ ]:
N = 16

In [ ]:
@variable(ising, s[1:N], Bin);

In [ ]:
J = randn(N,N);
J[ abs.(J) .< 2*0.67 ] .= 0;
A = map(J) do j
    if j==0
        0
    else
        1
    end
end

In [ ]:
count(x->x==0, J)/N^2

In [ ]:
@NLobjective(ising, Min, 
    sum( -J[i,j]*(2*s[i]-1)*(2*s[j]-1) for i=1:N, j=1:N)
);

In [ ]:
JuMP.optimize!(ising)

In [ ]:
function int2bin(x::Int; pad::Int=0)
    if x==0
        return zeros(Int, max(1, pad))
    end
    exponent = floor(Int, log2(x))+1
    m = max(exponent, pad)
    b = zeros(Int, m)
    for i=1+(m-exponent):exponent+(m-exponent)
        e = exponent-i+(m-exponent)
        b[i] = x÷(2^e)
        x -= b[i]*2^e
    end
    b
end

In [ ]:
H(s) = sum( -J[i,j]*(2*s[i]-1)*(2*s[j]-1) for i=1:N, j=1:N)

best = zeros(16)
bestH = H(best)
for i = 0:2^N-1
    local s = int2bin(i, pad=16)
    newH = H(s)
    if newH < bestH
        bestH = newH
        best = s
    end
end

In [ ]:
H(value.(s)), bestH

In [ ]:
[best value.(s)]

# SciML

Grown out of `DifferentialEquations.jl`, SciML bundles tools for

* solving differential equations (ODE, SDE, PDE, DDE, ...) `DifferentialEquations.jl`
* reaction networks `Catalyst`
* sensitivity analysis `DiffEqSensitivity`
* parameter optimization and "Neural DE" `DiffEqFlux`, `GalacticOptim`
* modeling language `ModelingToolkit`
* (...)

under one roof.

## An ODE

In [ ]:
using OrdinaryDiffEq

$$ \dot{u} = f(u,t) $$

In [ ]:
function anharm!(du, u, param, t)
    x,p = u
    m,a,b = param # V(x) = ax^4 + bx^2
    du[1] = p/m
    du[2] = -4a*x^3 - 2b*x
end

In [ ]:
V(x,p) = p[1]*x^4 + p[2]*x^2

In [ ]:
x0 = 0.9

In [ ]:
plot(range(-1,1,length=50), x->V(x, [1,-0.7]))
scatter!([x0], x->V(x, [1,-0.7]))

In [ ]:
anharm_prob = ODEProblem(anharm!, [x0,0.0], (0.0,10.0))

In [ ]:
anharm_sol = OrdinaryDiffEq.solve(anharm_prob, Tsit5(); p=[1.0,1.0,-0.7]);#[ [1.0]; popt]

In [ ]:
anharm_sol(1.0)

In [ ]:
plot(anharm_sol)
anharm_plot = scatter!(anharm_sol.t, getindex.(anharm_sol.u, 1), lab="")
#vline!([find_period(pdefault)], lab="")

In [ ]:
import FFTW

In [ ]:
fourier = FFTW.fft(getindex.(anharm_sol.(0:0.1:8),1))
fourier[ abs.(fourier) .< maximum(abs.(fourier)) ] .= 0+0im;
ifourier = real.(FFTW.ifft(fourier));

In [ ]:
unique(fourier)

In [ ]:
plot!(0:0.1:8, ifourier)

To find the period of oscillation, we integrate the system until the momentum reverses, i.e. $p(t_f) = 0$.

We use a `ContinuousCallback(cond, affect!)`. It applies when `cond==0` and is triggered when `cond` crosses from neg. to pos. or vice versa.
__Then it tracks back to the exact time point where the condition hit zero.__

`affect!` is a function to modify the integrator. Here it terminates integration. Other uses include affecting the current state, e.g. imparting momentum.

In [ ]:
function has_reversed(u, p, integrator)
    u[2]
end

function affect!(integrator)
    terminate!(integrator)
end

cb = ContinuousCallback(has_reversed, affect!)

In [ ]:
function find_period(p)
    anharm_sol = OrdinaryDiffEq.solve(anharm_prob, Tsit5(); tspan=(0.0,1000.0), p, callback=cb, sensealg=ForwardDiffSensitivity)
    return 2*anharm_sol.t[end]
end

In [ ]:
pdefault = [1.0, 1.0, -0.7]

In [ ]:
find_period(pdefault)

__Goal:__ Tune parameters to have $T=10$

$ \mathrm{solve} \to \mathrm{find\_period} \to \mathrm{floss} $

Need gradient $\partial_p \mathrm{floss} $

<hr/>

### Excursion: Automatic Differentiation

* https://mitmath.github.io/18337/lecture10/estimation_identification
* https://juliadiff.org/ChainRulesCore.jl/dev/index.html
* https://github.com/MikeInnes/diff-zoo
* https://github.com/JuliaDiff/ForwardDiff.jl/
* https://github.com/FluxML/Zygote.jl

Two ways to evaluate derivatives on a computer.

1. Finite differencing. 
2. Using information about primitive derivatives and the chain rule, aka AD.

Consider a sequence of functions $y = y_n\circ y_{n-1} \circ\cdots \circ y_1$. By the chain rule
$$ \frac{\partial y}{\partial x} = \frac{\partial x}{\partial x}\frac{\partial y_1}{\partial x}\frac{\partial y_2}{\partial y_1}\cdots\frac{\partial y_n}{\partial y_{n-1}}\frac{\partial y}{\partial y_n}\frac{\partial y}{\partial y} $$

Two ways to read the chain rule. Left to right (_forward mode_), right to left (_reverse mode, adjoint, backpropagation,..._).

<img src="img/forward_reverse.jpg" width=600/>

* __Forward mode__
   Keep track of derivatives during the forward pass (i.e. function evaluation). Easy to realize with _dual numbers_ $a+b\epsilon$ with $\epsilon^2=0$. Then set $f(a+b\epsilon) \equiv f(a) + bf'(a)\epsilon$ and observe that the chain rule is fulfilled.  
   Propagates _perturbations_ forward. In mathematical parlance, pushforward of tangent vector $v$, i.e. $v=e_i$.  In terms of Jacobians $Jv = J_nJ_{n-1}\cdots J_1 v$  
   Disadvantage: Need $N$ passed to determine all partial derivatives for $f: \mathbb{R}^N \to \mathbb{R}^M$, __but__ get derivatives of all $M$ components simultaneously.
   
   
* __Reverse mode__
   Make a forward pass and record the computational graph and intermediate values. Then traverse the graph backwards, propagating _sensitivities_ $\bar{x} \equiv \frac{\partial y}{\partial x}$ back (hence backpropagation).
   Mathematically this corresponds to pulling back the differential $dy = v_i\, dy^i$. Identify components $v_i$ with vector $v$ this yields a vector-Jacobian-product $(v^tJ_n)J_{n-1}\cdots J_1$, or transposing, $J_1^t\cdots J^t_{n-1}J^t_nv$, hence the name adjoint methods.  
   For a given output sensitivity, reverse mode AD computes partials for all inputs, __but__ needs $M$ passes to calculate all columns of the Jacobian.
  
   





#### Example:

In [ ]:
G(x) = x[1]^2 + x[2]^2

In [ ]:
G(x,y) = x^2 + y^2

In [ ]:
ForwardDiff.gradient(G, [1.0, 2.0])

In [ ]:
d = ForwardDiff.Dual{typeof(G), Float64, 1}(2.0, ForwardDiff.Partials{1, Float64}((1.0,)))

In [ ]:
G(1.0, d)

`ForwardDiff` overloads many functions with versions that accept and propagate dual numbers.

In [ ]:
methods(sin)

In [ ]:
DiffRules.diffrule(:Base, :sin, :x)

The same in reverse:

In [ ]:
Zygote.gradient(G, 1.0, 2.0)

<hr/>

### Learn parameters

In [ ]:
# Fix mass==1, only tune potential
floss(p) = forwarddiff(p) do p
    (find_period([ [1.0]; p]) - 10.0)^2
end

In [ ]:
pdefault

In [ ]:
floss(pdefault[2:end])

In [ ]:
Zygote.gradient(floss, pdefault[2:end])

#### Sensitivity Analysis

In [ ]:
using DiffEqSensitivity

In [ ]:
sense_prob = ODELocalSensitivityProblem(anharm!, [x0, 0.0], (0.0, 10.0), pdefault)

In [ ]:
sense_sol = OrdinaryDiffEq.solve(sense_prob, Tsit5())

In [ ]:
xs,dp = extract_local_sensitivities(sense_sol)

In [ ]:
xs'

In [ ]:
dp

In [ ]:
plot(xs', xlabel="time", ls=:dash, lab=["x" "p"])
plot!(dp[2]', lab=["dx" "dp"])

In [ ]:
using DiffEqFlux

In [ ]:
function train_cb(p, l)
    global iter
    if iter%10==0
        println("Loss: ", l)
        println("Parameter", p)
    end
    iter += 1
    return l < 1e-8
end

In [ ]:
global iter = 0
opt = ADAM(0.025)
popt = DiffEqFlux.sciml_train(floss, [1.0,-0.9], opt, maxiters=5000, cb=train_cb)

In [ ]:
find_period([ [1.0]; popt])

### Learn potential from scratch

In [ ]:
Fneural = Flux.Chain(
    Dense(1, 16, tanh),
    Dense(16,1)
    )

In [ ]:
Fneural([1.0])

In [ ]:
p, fnn = Flux.destructure(Fneural)

In [ ]:
fnn(p)([1.0])[1]

In [ ]:
jacobian(fnn(p), [1.0])

In [ ]:
function dudt!(du, u, param, t)
    x,p = u
    du[1] = p
    F = fnn(param)([x])[1]
    du[2] = F
end


In [ ]:
dudt!(zeros(2), [x0, 0.0], p, 0.0)

In [ ]:
ts = anharm_sol.t
ys = getindex.(anharm_sol.u, 1);

In [ ]:
prob = ODEProblem(dudt!, u0, tspan)

In [ ]:
function predict_osc(p)
    pr = remake(prob, p=p)
    OrdinaryDiffEq.solve(pr, Tsit5(), saveat=ts, sensealg=BacksolveAdjoint())
end

function loss_osc(p)
    sol = predict_osc(p)
    yhat = getindex.(sol.u, 1)
    sum(abs2, yhat .- ys)
end

In [ ]:
predict_osc(p)

In [ ]:
loss_osc(p)

In [ ]:
Flux.gradient(loss_osc, p)

In [ ]:
pmin = @time DiffEqFlux.sciml_train(loss_osc, p, ADAM(0.025), maxiters=100)

In [ ]:
predy = map(range(-2, 2, length=50)) do x
    fnn(pmin)([x])[1]
end

In [ ]:
pmin.minimum

In [ ]:
plot(anharm_sol)
plot!(predict_osc(pmin))

In [ ]:
plot(range(-2, 2, length=50), predy)
plot!(range(-2, 2, length=50), x->Flux.gradient(x->V(x, pdefault), x)[1])

## A stochastic SDE

In [ ]:
using StochasticDiffEq

$$ dX_t = f(X_t,t)dt + g(X_t,t)dW_t $$

In [ ]:
function hooke!(du, u, p, t)
    k = p[1]
    x = u[1]
    v = u[2]
    du[1] = v
    du[2] = -k*x
end

function add_noise!(du, u, p, t)
    du[1] = zero(p[2])
    du[2] = p[2]*u[1]
end

tspan = (0f0, 10f0)
u0 = [1f0, 0f0]

osc_prob = SDEProblem{true}(hooke!, add_noise!, u0, tspan)

In [ ]:
sol = solve(osc_prob, SOSRI(), p=[2f0,1f0]);

In [ ]:
Plots.plot(sol, seriestype=[:line :line])

In [ ]:
osc_ens_prob = EnsembleProblem(osc_prob)

In [ ]:
@time ens_sol = StochasticDiffEq.solve(osc_ens_prob, SOSRI(), p=[2f0,1f0], trajectories=10_000, saveat=range(tspan..., length=25));

In [ ]:
truem = mean(ens_sol, dims=3)[1,:,1]
truev = var(ens_sol, dims=3)[1,:,1];

In [ ]:
gr()
default(fmt=:png)

In [ ]:
summary = EnsembleSummary(ens_sol)

In [ ]:
Plots.plot(summary, vars=(0,))

In [ ]:
plot(summary.t, truem, ribbon=sqrt.(truev), ribbon_alpha=0.1, lw=4)

In [ ]:
using StatsBase

In [ ]:
function osc_predict(p; n=100)
    prob = remake(osc_prob, u0=p[1:2], p=p[3:end])
    ens_prob = EnsembleProblem(prob)
    StochasticDiffEq.solve(ens_prob, SOSRI(), trajectories=n, saveat=range(tspan..., length=25), sensealg=ForwardDiffSensitivity)
end

In [ ]:
function osc_loss(p; kwargs...)
    osc_predict(p; kwargs...)
    m = mean(sol, dims=3)[1,:,1]
    v = var(sol, dims=3)[1,:,1]
    sum(abs2, truem .- m) + 0.1*sum(abs2, truev .- v), sol
end

In [ ]:
p=[u0; [2f0,1f0]]
remake(osc_prob, u0=p[1:2], p=p[3:end])

In [ ]:
using ForwardDiff

using DiffEqSensitivity

In [ ]:
ForwardDiff.gradient(p->osc_loss(p)[1], p)

In [ ]:
ForwardDiff.jacobian(osc_predict, p)

In [ ]:
Plots.plot!(t->cos(sqrt(2)*t))

## Reaction Networks

In [ ]:
using Catalyst

In [ ]:
rn = @reaction_network begin
    α, S + I --> 2I
    β, I --> R
end α β

In [ ]:
u0 = 
sprob = SDEProblem(rn, u0, tspan, p)
ssol  = solve(sprob, EM(), dt=.01)